# Heart rate detection

### Importing the dependencies

In [224]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ifxdaq
import processing
import numpy as np
from ifxdaq.sensor.radar_ifx import RadarIfxAvian

### Accessing the json configuration file

In [225]:
config_file = "radar_configs/RadarIfxBGT60.json"
raw_data    = []

### Data collection and live detection of the heart rate for a quasi-static person 

In [252]:
raw_data = []       
time = 15000
with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        
        if(len(raw_data) > time-1 and len(raw_data) % time == 0):  
            
            # 5000 is the number of frames. which corresponds to 5seconds
        
            data = np.swapaxes(np.asarray(raw_data), 0, 1)
            
            phases, abses, _, _,st_data = processing.do_processing(data)       # preprocessing to get the phase information
            range_data, range_idx = processing.processing_rangeData(data)
            
            st_data = np.abs(st_data)

            op = np.average(st_data,axis=0)
            
            fil = st_data[1,:]                                           #filtering the data using a bandpass butterworth filter
            filter = scipy.signal.butter(N = 7, btype='bandpass', Wn= [0.8/500,2/500], fs = 2, output = 'sos')
            filtered = scipy.signal.sosfilt(filter, fil)

            a = scipy.signal.find_peaks(filtered)[0]

            heart_rate = ((len(a)*60)/15)                                   # heart rate calculation

            print("Your heart rate is " + str(heart_rate) + "bpm.")
            if((heart_rate >=60) and (heart_rate<=100)):
                print("You are resting now.")
            else:
                print("You are anxious now. Calm down!")
            
            raw_data = []

Your heart rate is 100.0bpm.
You are resting now.
Your heart rate is 96.0bpm.
You are resting now.
Your heart rate is 84.0bpm.
You are resting now.
Your heart rate is 92.0bpm.
You are resting now.
Your heart rate is 100.0bpm.
You are resting now.
Your heart rate is 92.0bpm.
You are resting now.
Your heart rate is 80.0bpm.
You are resting now.
Your heart rate is 84.0bpm.
You are resting now.
Your heart rate is 104.0bpm.
You are anxious now. Chill down!
Your heart rate is 80.0bpm.
You are resting now.
Your heart rate is 92.0bpm.
You are resting now.
Your heart rate is 84.0bpm.
You are resting now.
Your heart rate is 96.0bpm.
You are resting now.
Your heart rate is 92.0bpm.
You are resting now.
Your heart rate is 88.0bpm.
You are resting now.
Your heart rate is 76.0bpm.
You are resting now.
Your heart rate is 84.0bpm.
You are resting now.
Your heart rate is 84.0bpm.
You are resting now.
Your heart rate is 92.0bpm.
You are resting now.
Your heart rate is 76.0bpm.
You are resting now.
Your 

ErrorCommunicationError: communication error between host and device (IFX_ERROR_COMMUNICATION_ERROR)